# **update_collection**

---

<br><br><br><br>

## **Objectives**

---

Pipeline process for taking Helvault .csv files as inputs and storing them in a master "collection_db.csv" 

<br>

- Input csv files from Helvault
    - which format from helvault gives us the most info (we can filter out whatever isn't necessary)
    - best format for output: Helvault Pro CSV
    - import entire owned cards collection from Helvault 
    - How do we join in the deck / container that a given card is in?
    - Can we export every individual container from Helvault at once to the process all .csv files at once?

    - what is the best place to export the collection into for easy python ingestion?
        1. Save to OneDrive (as in between staging) 
        2. copy from OneDrive to git folder for Helvault 
        - directly to git folder on phone?

<br>

- Output .csv file (collection_db.csv)
    - export collection_db.csv to [text](https://)
    - What is the best format?
        - Primary keys
            - scryfall_id
            - name / front
        - other useful data points
            - back (for mdfcs)
            - price
            - set code (3 alpha numeric)
            - color
            - type
            - proxy_ind (bool)
                - use a placeholder status code from Helvault

<br><br><br><br>

## **TO DOs**

---

- TODO [ ] `update_collection`: convert input/output processing and storage of collection_db to .parquet file format: 2022-09-25
    - all other data formats can remain as .csvs for now.

<br><br><br><br><br><br>

## **Imports / Setup Environment**
---

In [1]:
import pandas as pd
from data_mgmt_fp import eda, fcal, notifier

### **Variable Setup**

In [2]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H-%M-%S")

In [3]:
input_helvault_owned_nm = 'helvaultPro.csv'
input_helvault_owned_dir = 'C:\\git\\mtg-proj\\data\\input\\helvault_csv\\owned_cards_input\\'
input_helvault_owned_path = input_helvault_owned_dir+input_helvault_owned_nm

input_helvault_cntnr_dir = 'C:\\git\\mtg-proj\\data\\input\\helvault_csv\\'
input_helvault_cntnr_path = input_helvault_cntnr_dir+''

archive_git_nm = 'collection_db_arch_'+dt_string+'.csv'
archive_git_dir = 'C:\\git\\mtg-proj\\data\\output\\collection_db\\.archive'
archive_git_path = archive_git_dir+'\\'+archive_git_nm

output_csv_nm = 'collection_db.csv'
output_csv_dir = 'C:\\git\\mtg-proj\\data\\output\\collection_db\\'
output_csv_path = output_csv_dir+output_csv_nm

<br><br><br><br>

## **Data Ingestion/Preprocessing**

---

In [4]:
input_owned_df = pd.read_csv(input_helvault_owned_path)

In [5]:
# create index_col
input_owned_df['index_col'] = input_owned_df.index

input_owned_df = input_owned_df.rename(columns={"estimated_price": "estimated_price_upd"}, errors="raise")

# re-order colms
input_owned_df = input_owned_df.reindex([
    'scryfall_id',
    'estimated_price_upd'
], axis=1)

In [6]:
#import all csv files at once and glue them together
import glob

directoryPath = input_helvault_cntnr_path

glued_data_df = pd.DataFrame()
for file_name in glob.glob(directoryPath+'*.csv'):
    df = pd.read_csv(file_name, low_memory=False)

    cntnr_txt_var = file_name.rsplit('\\', 1)[-1] or file_name
    cntnr_txt_var = cntnr_txt_var.split('.')[0] or cntnr_txt_var

    cntnr_type_var = cntnr_txt_var.split('-')[0] or cntnr_txt_var
    cntnr_name_var = cntnr_txt_var.split('-')[1] or cntnr_txt_var

    df['cntnr_type'] = cntnr_type_var
    df['cntnr_name'] = cntnr_name_var
    glued_data_df = pd.concat([glued_data_df,df],axis=0)

glued_data_df=glued_data_df.sort_values(by=['set_code','collector_number'])

glued_data_df=glued_data_df.reset_index()
glued_data_df['index_col'] = glued_data_df.index

In [7]:
collection_db_df_stg = glued_data_df
collection_db_df_stg = collection_db_df_stg.rename(columns={"estimated_price": "price_est_usd"}, errors="raise")
collection_db_df_stg = collection_db_df_stg.rename(columns={"quantity": "count_owned"}, errors="raise")

#column reordering for readability
collection_db_df_stg = collection_db_df_stg.reindex([
    'index_col',
    'scryfall_id',
    'oracle_id',
    'name',
    'extras',
    'cntnr_type',
    'cntnr_name',
    'set_code',
    'set_name',
    'collector_number',
    'rarity',
    'language',
    'count_owned',
    'price_est_usd'
], axis=1)

<br><br><br><br>

## **Scrython Data Ingestion**

---

In [8]:
# [x] add data pull from scryfall
import nest_asyncio
import scrython

scryfall_id_list = collection_db_df_stg['scryfall_id'].tolist()

price_list = []
index = len(scryfall_id_list)

# ====================================

nest_asyncio.apply()
i = 0

print('starting scryfall api search for prices; might take a few minutes')
for sf_id in scryfall_id_list:
    card = scrython.cards.Id(id=sf_id)
    card_price_str = card.prices('usd')
    price_list.append(card_price_str)
    i = i + 1
    progress_stat = str(round((i / index),2)) + ' / 1.00'
    print(progress_stat, end='\r')
    # print(card_price_str, end="            ")
print('done searching scryfall api; no errors')

collection_db_df_stg['price_est_usd'] = price_list

# ====================================

starting scryfall api search for prices; might take a few minutes
done searching scryfall api; no errors


<br><br><br><br>

## **Cleaning / Transform**

---

In [9]:
import numpy as np

# transform set_code to be all upper case 
collection_db_df_stg['set_code'] = collection_db_df_stg['set_code'].str.upper()

# add front and back fields for MPC input
# new data frame with split value columns
split_df = collection_db_df_stg["name"].str.split("//", n = 1, expand = True)
# making separate first name column from new data frame
collection_db_df_stg["front"]= split_df[0]
# making separate last name column from new data frame
collection_db_df_stg["back"]= split_df[1]

# cleaning count owned
collection_db_df_stg['count_owned']=collection_db_df_stg['count_owned'].fillna(0)
collection_db_df_stg['count_owned']=collection_db_df_stg['count_owned'].astype(int)
# cleaning price
collection_db_df_stg['price_est_usd'] = collection_db_df_stg['price_est_usd'].astype(float)

# lower limit of $0 on estimated_price field
collection_db_df_stg['price_est_usd'] = collection_db_df_stg['price_est_usd'].clip(0,1000000)


collection_db_df_stg['extras']=collection_db_df_stg['extras'].astype(str)
# ====================================
# [x] converting extras to 'alteredArt'
collection_db_df_stg['extras'] = np.where(
    collection_db_df_stg['cntnr_name'].str.find('Proxy') != -1,
    'alteredArt',
    collection_db_df_stg['extras']
);

collection_db_df_stg['extras'] = collection_db_df_stg['extras'].apply(
    lambda x: x.replace('misprint', 'alteredArt')
);

# collection_db_df_stg['extras'] = collection_db_df_stg['extras_s2']
# collection_db_df_stg['extras'] = collection_db_df_stg['extras_s1']

In [10]:
#final column reordering
collection_db_df_stg = collection_db_df_stg.reindex([
    'index_col',
    'oracle_id',
    'scryfall_id',
    'name',
    'front',
    'back',
    'extras',
    'cntnr_type',
    'cntnr_name',
    'set_code',
    'set_name',
    'collector_number',
    'rarity',
    'language',
    'count_owned',
    'price_est_usd'
], axis=1)

In [11]:
eda.copi_df(collection_db_df_stg)
# collection_db_df_stg

<br><br><br><br>

## **Outputs**

---

In [12]:
collection_db_df_stg.to_csv(output_csv_path,index=False)

### **Save to Archive**

In [13]:
collection_db_df_stg.to_csv(archive_git_path,index=False)

In [ ]:
#stop